In [171]:
import nltk
import pandas as pd
from textblob import TextBlob
import os
import numpy as np
from os import path
from PIL import Image
import re

In [172]:
#import scraped articles
df = pd.read_csv('abovethelaw_articles.csv')

In [173]:
#perform quick anlysis
df.shape

(3490, 7)

In [174]:
df.dtypes

Unnamed: 0       int64
author          object
publish_date    object
sub_title       object
summary         object
tags            object
title           object
dtype: object

In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3490 entries, 0 to 3489
Data columns (total 7 columns):
Unnamed: 0      3490 non-null int64
author          3490 non-null object
publish_date    2544 non-null object
sub_title       3489 non-null object
summary         3220 non-null object
tags            3487 non-null object
title           3490 non-null object
dtypes: int64(1), object(6)
memory usage: 190.9+ KB


In [176]:
df.sample(10)

,Unnamed: 0,author,publish_date,sub_title,summary,tags,title
568,569,Joe Patrice,NaN,* Alabama fan allegedly shot despicable fronts...,* Alabama fan allegedly shot despicable front ...,"Americans with Disabilities Act,Auburn,Barack ...",Non-Sequiturs: 12.03.13
3087,3088,Kathryn Rubino,"Nov 10, 2017",Congratulations are in order.,Tuesday was Election Day — the first since Don...,"Biglaw,Election Day,Jenny Durkin,Justin Fairfa...",The Biglaw Attorneys Who Won Big On Election N...
218,219,Kathryn Rubino,"Dec 21, 2017","A fully paid, gender-neutral parental leave po...","In what is becoming a, yet another firm has ma...","Biglaw,Parental Leave,Skadden Arps,The Jabot",Biglaw Firm Gets With The Times And Makes Big ...
1396,1397,Mark Herrmann,"Sep 29, 2014",What questions should clients ask during beaut...,How do you think we pick lawyers to defend us ...,"Advice,Beauty Contests,Biglaw,In-House Counsel...",How Corporations Actually Choose Outside Counsel
3401,3402,Staci Zaretsky,"Jul 23, 2018",Did everyone at your firm receive an offer? Le...,We’ve had an incredibly slow season in terms o...,"Biglaw,Cold Offers,Law Schools,Law Students,No...",Summer Associate Offer Rates (2018): Which Fir...
707,708,Staci Zaretsky,NaN,"You'll want to keep reading, because this is t...","About two years ago, in May 2012, filed for b...","Biglaw,Canada,Dissolution,DLA Piper,Heenan Bla...","Another ‘Storied’ Biglaw Firm Fails, Inciting ..."
3262,3263,Kathryn Rubino,"Aug 20, 2018",The judge denied the defense's request for no ...,"On Friday, Eastern District of New York Distri...","Biglaw,Evan Greebel,Katten Muchin Rosenman,Kay...",Former Biglaw Partner Gets 18 Months In Prison...
2743,2744,Joe Patrice,"May 16, 2017",Reportedly around 40 attorneys are being laid ...,Law firms may be giving off a sense of “cautio...,"2016 Associate Salary Increase,Biglaw,Layoffs,...",HUGE LAYOFF REPORT: Biglaw Firm Lays Off Assoc...
3471,3472,Staci Zaretsky,"Jun 28, 2018",Size matters in the legal profession.,There are many different and exciting ways to ...,"Biglaw,NLJ 500,Rankings",The 500 Largest Law Firms In America (2018)
2373,2374,Kathryn Rubino,"Oct 14, 2016",It looks like the Biglaw firm might be right.,Traci Ribeiro may not get her day in court. Th...,"Biglaw,Employment law,Gender Discrimination,Pa...",Judge Really Bummed Biglaw Discrimination Suit...


In [177]:
# Fill na with empty string
df['summary'] = df['summary'].fillna('')

In [178]:
# Combine title and summary cols into summary
df['summary'] = df['title'] + ' ' + df['summary']

In [180]:
# Convert all the string to lower cases
df['summary'] = df['summary'].str.lower()
# Remove any URLs in summary col
df['summary'] = df['summary'].apply(lambda x: re.sub('http\S*', '', x))
# Remove empty space in summary col
df['summary'] = df['summary'].apply(lambda x: re.sub('\s+', ' ', x))
# Remove all rows that are empty
df = df.loc[df['summary'] != ""]
# Remove punctuation
df['summary'] = df['summary'].apply(lambda x: re.sub('[^\w\s]', '', x))

In [199]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [184]:
from wordcloud import WordCloud

In [189]:
# Import balance jpg for word cloud
d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
scale_mask = np.array(Image.open(path.join(d, "weight-balance.jpg")))

In [190]:
wc = WordCloud(background_color="white", max_words=2000, mask=scale_mask, max_font_size=300, random_state=42)
# generate word cloud
wc.generate(''.join(df['summary']))
# export word cloud to png file
wc.to_file("outputs/balance.png")

In [192]:
# Run sentiment analysis and save to csv for R analysis 

def sentiment_func(x):
    sentiment = TextBlob(x['summary'])
    x['polarity'] = sentiment.polarity
    x['subjectivity'] = sentiment.subjectivity
    return x

sample = df.apply(sentiment_func, axis=1).to_csv('outputs/abovethelaw_sentiment.csv',index=False)

In [194]:
# preprosessing for tags column
df['tags'] = df['tags'].fillna('')
# Convert all the string to lower cases
df['tags'] = df['tags'].str.lower()
# Remove any URLs in summary col
df['tags'] = df['tags'].apply(lambda x: re.sub('http\S*', '', x))
# Remove empty space in summary col
df['tags'] = df['tags'].apply(lambda x: re.sub('\s+', ' ', x))
# Remove all rows that are empty
df = df.loc[df['tags'] != ""]
# Remove punctuation
df['tags'] = df['tags'].apply(lambda x: re.sub('[^\w\s]', '  ', x))

In [198]:
# Import tag icon for word cloud
d_ = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
scale_mask = np.array(Image.open(path.join(d_, "tag_icon.jpg")))

wc = WordCloud(background_color="white", max_words=2000, mask=scale_mask, max_font_size=300, random_state=42)
# generate word cloud
wc.generate(''.join(df['tags']))
# save word cloud to png
wc.to_file("outputs/tag.png")